## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
import nltk
from nltk.corpus import stopwords
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix

RANDOM_STATE = 42

## Read-In Data

In [2]:
subreddits = pd.read_csv('../data/subreddits_preprocessed.csv')
subreddits.drop(columns = 'Unnamed: 0', inplace = True)

In [3]:
subreddits.head(2)

,title,selftext,subreddit,author,num_comments,score,timestamp,original_text,post_length_char,post_length_words,is_unethical,stemmer_text,polarity,sentiment_cat
0,: Answers to why,NaN,LifeProTips,AlienAgency,2,1,2020-07-17,: Answers to why,16,4,0,: answer to whi,0.0,Neutral
1,¿Quieres obtener juegos y premios gratis en tu...,NaN,LifeProTips,GarbageMiserable0x0,2,1,2020-07-17,¿Quieres obtener juegos y premios gratis en tu...,60,10,0,¿quier obten juego y premio grati en tu tiempo...,0.0,Neutral


## Model Preparation

In a separate set of models, I determined that stemmed text and the Tfidf Vectorizer would be a good choice for my data. Therefore, I will conduct a train test split on the stemmed text and set up a Column Transformer to only vectorize my text data.

### Train Test Split

In [4]:
#features2 = ['num_comments', 'score', 'post_length_char', 'post_length_words', 'polarity', 'stemmer_text']
features = ['stemmer_text']
X = subreddits[features]
y = subreddits['is_unethical']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = RANDOM_STATE, stratify = y)

### Define Custom Stop Words Hyperparameter for Vectorizer

In [6]:
custom_stop_words = stopwords.words('english') + ['ulpt', 'lpt']

### Build Column Transformer to Only Apply Vectorizer to Text Features

In [7]:
tfidf_transformer = ColumnTransformer([
    ('tfidf', TfidfVectorizer(stop_words = 'english'), 'stemmer_text'),], 
    remainder='passthrough')

## Modeling

MARKDOWN TO DESCRIBE THE PROCESS!

### Functions

In [8]:
def display_accuracy_scores(model, xtrain, ytrain, xtest, ytest):
    print(f'The cross validation accuracy score is {cross_val_score(model, xtrain, ytrain).mean()}.')
    print(f'The training accuracy score is {model.score(xtrain, ytrain)}.')
    print(f'The testing accuracy score is {model.score(xtest, ytest)}.')

In [9]:
def display_accuracy_scores_gs(model, xtrain, ytrain, xtest, ytest):
    print(f'The training accuracy score is {model.score(xtrain, ytrain)}.')
    print(f'The testing accuracy score is {model.score(xtest, ytest)}.')

In [10]:
def get_sensitivity(actual_values, predicted_values):
    tn, fp, fn, tp = confusion_matrix(actual_values, predicted_values).ravel()
    return tp/(tp+fn)
    

### Model 1: Null Model

In [11]:
null = DummyClassifier(strategy = 'stratified')

In [12]:
null.fit(X_train, y_train);

In [13]:
display_accuracy_scores(model = null, xtrain = X_train, xtest = X_test, ytrain = y_train, ytest = y_test)

The cross validation accuracy score is 0.504381034431538.
The training accuracy score is 0.5194931773879142.
The testing accuracy score is 0.49829545454545454.


In [14]:
get_sensitivity(y_test, null.predict(X_test))

0.5083507306889353

In order to perform better than the null model, any model that I build will need to perform better than 50.8% accuracy on the testing data.

### Model 2a: Logistic Regression with No Regularization

#### Create Pipeline

In [15]:
logreg_pipe = Pipeline([
    ('tfidf', tfidf_transformer),
    ('logreg', LogisticRegression(penalty = 'none', solver = 'newton-cg', max_iter = 600))
])

# The model would not converge for the other solvers. Newton-cg can be used to fit larger datasets.

#### Grid Search Over Pipeline

In [16]:
logreg_pipe_params = {
    'tfidf__tfidf__ngram_range': [(1,1)],
    'tfidf__tfidf__min_df': [5],
    'tfidf__tfidf__max_df': [0.98]
}

# Note: All other hyperparameter options removed

In [17]:
gs_logreg_pipe = GridSearchCV(logreg_pipe, param_grid = logreg_pipe_params, cv = 5, verbose = 1, n_jobs = -1)

In [18]:
gs_logreg_pipe.fit(X_train, y_train);

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.1s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.1s finished


In [19]:
gs_logreg_pipe.best_params_

{'tfidf__tfidf__max_df': 0.98,
 'tfidf__tfidf__min_df': 5,
 'tfidf__tfidf__ngram_range': (1, 1)}

The best parameters for this model were determined to be a maximum occurrence in the data frame of 0.98, a minimum occurrence of 5, and and ngram range of 1.

#### Evaluate Accuracy Metric

In [20]:
gs_logreg_pipe.best_score_

0.7141775348325956

In [21]:
display_accuracy_scores_gs(model = gs_logreg_pipe, xtrain = X_train, xtest = X_test, ytrain = y_train, ytest = y_test)

The training accuracy score is 0.9982943469785575.
The testing accuracy score is 0.7323863636363637.


In [23]:
get_sensitivity(y_test, gs_logreg_pipe.predict(X_test))

0.7463465553235908

Although this model performs better than baseline accuracy, this model is extremely overfit, but this is likely due to the large number of features in the model without any regularization. (Note: In this model, there are 3035 features (3030 are words and 5 are numerical features).

### Model 2b: Logistic Regression with Regularization

#### Create Pipeline

In [81]:
logreg_reg_pipe = Pipeline([
    ('tfidf', tfidf_transformer),
    ('ss', StandardScaler(with_mean = False)),
    ('logreg', LogisticRegression())])

#### Grid Search Over Pipeline

In [73]:
logreg_reg_pipe_params = {
    'tfidf__tfidf__ngram_range': [(1,2)],
    'tfidf__tfidf__max_df': [0.90],
    'tfidf__tfidf__min_df': [2],
    'logreg__penalty': ['l2'],
    'logreg__C': [0.0001, 0.00001, 0.000001],
    'logreg__solver': ['liblinear']
}
# Only best params remain in grid

In [74]:
gs_logreg_reg_pipe = GridSearchCV(logreg_reg_pipe, param_grid = logreg_reg_pipe_params, cv = 5, verbose=1, n_jobs = -1 )

In [75]:
gs_logreg_reg_pipe.fit(X_train, y_train);

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/jocelynlutes/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.6s finished


In [28]:
gs_logreg_reg_pipe.best_params_

{'logreg__C': 0.0001,
 'logreg__penalty': 'l2',
 'logreg__solver': 'liblinear',
 'tfidf__tfidf__max_df': 0.9,
 'tfidf__tfidf__min_df': 2,
 'tfidf__tfidf__ngram_range': (1, 2)}

#### Evaluate Accuracy Metric

In [77]:
print(f'The cross val score is {round(gs_logreg_reg_pipe.best_score_, 4)}.')

The cross val score is 0.5441.


In [78]:
display_accuracy_scores_gs(model = gs_logreg_reg_pipe, xtrain = X_train, xtest = X_test, ytrain = y_train, ytest = y_test)

The training accuracy score is 0.544103313840156.
The testing accuracy score is 0.5443181818181818.


In [79]:
get_sensitivity(y_test, gs_logreg_reg_pipe.predict(X_test))

1.0

Although this model has improved accuracy by about 8%, it is still very overfit to the training data. Reducing features in the vector of words and decreasing the C value even further do not seem to help. 

### Model 3: Multinomial Naive Bayes

#### Create Pipe

In [32]:
nb_pipe = Pipeline([
    ('tfidf', tfidf_transformer),
    ('nb', MultinomialNB())
])

#### Grid Search Over Pipe

In [33]:
nb_pipe_params = {
    'tfidf__tfidf__max_features': [4000],
    'tfidf__tfidf__min_df': [2],
    'tfidf__tfidf__max_df': [0.9],
    'tfidf__tfidf__ngram_range': [(1,1)],
    'nb__alpha':[1]
}

In [34]:
gs_nb_pipe = GridSearchCV(nb_pipe, param_grid = nb_pipe_params, cv = 5, verbose = 1, n_jobs = -1)

In [35]:
gs_nb_pipe.fit(X_train, y_train);

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [36]:
gs_nb_pipe.best_params_

{'nb__alpha': 1,
 'tfidf__tfidf__max_df': 0.9,
 'tfidf__tfidf__max_features': 4000,
 'tfidf__tfidf__min_df': 2,
 'tfidf__tfidf__ngram_range': (1, 1)}

#### Evaluate Sensitivity Metric

In [37]:
get_sensitivity(y_test, gs_nb_pipe.predict(X_test))

0.8716075156576201

### Model 4: KNN

#### Create Pipe

In [59]:
knn_pipe_robust = Pipeline([
    ('tfidf', tfidf_transformer),
    ('rs', RobustScaler(with_centering = False)),
    ('knn', KNeighborsClassifier())
])

# In this case, robust scaler performed better than standard scaler

#### Grid Search Over Pipe

In [82]:
knn_pipe_params = {
    'tfidf__tfidf__min_df': [2],
    'tfidf__tfidf__max_df': [0.9],
    'tfidf__tfidf__ngram_range': [(1,1)],
    'knn__n_neighbors' : [15],
    'knn__metric':['euclidean'],
    'knn__weights': ['distance']    
}

In [83]:
gs_knn_pipe_robust = GridSearchCV(knn_pipe_robust, knn_pipe_params, cv = 5, verbose = 1, n_jobs = -1)

In [84]:
gs_knn_pipe_robust.fit(X_train, y_train);

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.1s finished


In [85]:
gs_knn_pipe_robust.best_params_

{'knn__metric': 'euclidean',
 'knn__n_neighbors': 15,
 'knn__weights': 'distance',
 'tfidf__tfidf__max_df': 0.9,
 'tfidf__tfidf__min_df': 2,
 'tfidf__tfidf__ngram_range': (1, 1)}

#### Evaluate Sensitivity Metric

In [71]:
gs_knn_pipe_robust.best_score_

0.7273372745907727

In [70]:
get_sensitivity(y_test, gs_knn_pipe_robust.predict(X_test))

0.6711899791231732

### Model 5: Decision Tree

#### Create Pipeline

In [86]:
dt_pipe = Pipeline([
    ('tfidf', tfidf_transformer),
    ('dt', DecisionTreeClassifier())
])

#### Grid Search Over Pipeline

In [119]:
dt_pipe_params = {
    'tfidf__tfidf__min_df': [2, 3, 4],
    'tfidf__tfidf__max_df': [0.9, 0.95, 0.98],
    'tfidf__tfidf__ngram_range': [(1,1), (1,2)],
    'dt__min_samples_split': [9],
    'dt__min_samples_leaf': [1]
}

In [120]:
gs_dt_pipe = GridSearchCV(dt_pipe, param_grid = dt_pipe_params, cv = 5, verbose = 1, n_jobs = -1)

In [121]:
gs_dt_pipe.fit(X_train, y_train);

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
/Users/jocelynlutes/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    9.2s finished


In [109]:
gs_dt_pipe.best_params_

{'dt__min_samples_leaf': 1,
 'dt__min_samples_split': 9,
 'tfidf__tfidf__max_df': 0.98,
 'tfidf__tfidf__min_df': 3,
 'tfidf__tfidf__ngram_range': (1, 1)}

#### Evaluate Sensitivity Metric

In [122]:
get_sensitivity(y_test, gs_dt_pipe.predict(X_test))

0.6889352818371608

In [123]:
get_sensitivity(y_train, gs_dt_pipe.predict(X_train))

0.9811912225705329

In [124]:
gs_dt_pipe.best_score_

0.6803101512135706

In [125]:
gs_dt_pipe.score(X_train, y_train)

0.9788011695906432

### Model 6: Bagging Classifier

#### Create Pipe

In [130]:
bag_pipe = Pipeline([
    ('tfidf', tfidf_transformer),
    ('bc', BaggingClassifier())
])

In [131]:
bag_pipe_params = {
    'tfidf__tfidf__min_df': [2, 3, 4],
    'tfidf__tfidf__max_df': [0.9, 0.95, 0.98],
    'tfidf__tfidf__ngram_range': [(1,1), (1,2)],
}

In [133]:
gs_bag_pipe = GridSearchCV(bag_pipe, param_grid = bag_pipe_params, cv = 5, verbose = 1, n_jobs = -1)

In [134]:
gs_bag_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   47.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('tfidf',
                                                                         TfidfVectorizer(analyzer='word',
                                                                                         binary=False,
                                                                                         decode_error='strict',
                                                                                         dtype=<class 'numpy.float64'>,
                             

In [135]:
gs_bag_pipe.best_score_

0.7136941861501441

In [137]:
get_sensitivity(y_test, gs_bag_pipe.predict(X_test))

0.7004175365344467